In [1]:
import torch
import torch_geometric as pyg
import numpy as np


/home/jrm28/.conda/envs/gelato/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
{key: [1] for key in np.unique(r)}

{0: [1],
 1: [1],
 2: [1],
 3: [1],
 4: [1],
 5: [1],
 6: [1],
 7: [1],
 8: [1],
 9: [1],
 10: [1],
 11: [1]}

In [53]:
l = np.array([[0, 1], [1, 4], [1, 5], [1, 6], [0, 2], [2, 7], [0, 3], [3, 8], [3, 9], [3, 10], [3, 11]]).transpose()
r = np.array([l[1, :], l[0, :]])
edge_index = torch.from_numpy(r)

train_loader = pyg.loader.NeighborSampler(edge_index,
                               sizes={key: 1 for key in np.unique(r)}, batch_size=1)


# train_loader = pyg.loader.NeighborLoader(edge_index,
#                                 num_neighbors={key: [1] * 2 for key in np.unique(r)}, batch_size=1, input_nodes=None)
                            
                               

In [62]:
train_loader.sample([0])

(1,
 tensor([ 0,  1,  3,  5,  4,  2,  6, 11, 10,  8,  9,  7]),
 [EdgeIndex(edge_index=tensor([[ 1,  2,  5,  3,  4,  6,  7,  8,  9, 10, 11],
          [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  5]]), e_id=tensor([ 0,  6,  4,  2,  1,  3, 10,  9,  7,  8,  5]), size=(12, 12)),
  EdgeIndex(edge_index=tensor([[ 1,  2,  5,  3,  4,  6,  7,  8,  9, 10, 11],
          [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  5]]), e_id=tensor([ 0,  6,  4,  2,  1,  3, 10,  9,  7,  8,  5]), size=(12, 12)),
  EdgeIndex(edge_index=tensor([[ 1,  2,  5,  3,  4,  6,  7,  8,  9, 10, 11],
          [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  5]]), e_id=tensor([ 0,  6,  4,  2,  1,  3, 10,  9,  7,  8,  5]), size=(12, 12)),
  EdgeIndex(edge_index=tensor([[ 1,  2,  5,  3,  4,  6,  7,  8,  9, 10, 11],
          [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  5]]), e_id=tensor([ 0,  6,  4,  2,  1,  3, 10,  9,  7,  8,  5]), size=(12, 12)),
  EdgeIndex(edge_index=tensor([[ 1,  2,  5,  3,  4,  6,  7,  8,  9, 10, 11],
          [ 0,  0,  

In [24]:
def new_degree(edge_index, num_nodes, out=True):
    return pyg.utils.degree(edge_index[0 if out else 1], num_nodes)   


In [28]:
len(np.unique(r))

12

In [32]:
1 / new_degree(edge_index, len(np.unique(r)), False)

tensor([0.3333, 0.3333, 1.0000, 0.2500,    inf,    inf,    inf,    inf,    inf,
           inf,    inf,    inf])